In [57]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import numpy as np


In [28]:
df = pd.read_csv('test_full.csv')

mn = df.ts.min()
mx = df.ts.max()
#df['hour'] = df.ts % (60*60*24)
df['day'] = (df.ts-mn)  // (60*60*24) #% (60*60*24*7)
print( df.shape )
df.head()
print(mn)
print(mx)
day=(mx-mn)// (60*60*24)
print(day)

(300000, 6)
1659304800
1661723997
27


In [29]:
AIDS = df.aid.unique()
DAYS = df.day.unique()

In [30]:
DAYS
print(len(DAYS))

28


In [31]:
AIDS
print(len(AIDS))

112119


In [32]:
tmp = df.groupby(['aid','day']).agg({'aid':'count','session':'count'})
tmp.columns = ['item','user']

# item & user
tmp = df.groupby(['aid','day']).agg({'aid':'count','session':'count'})
tmp.columns = ['item','user']
aid
其中 'item' 列表示每个 aid 和 day 的组内一共有多少个不同的 aid，即不同的物品（items）；'user' 列表示每个 aid 和 day 的组内有多少个不同的用户（users），即参加会话的用户数量。这一步产生的tmp统计的是一个物品一天中被多少用户访问了

In [33]:
tmp

item  user
aid     day            
4       16      1     1
        21      2     2
38      0       1     1
        1       1     1
        3       1     1
...           ...   ...
1855547 3       1     1
1855584 0       1     1
1855594 0       2     2
1855597 2       1     1
1855600 11      1     1

[181588 rows x 2 columns]

In [34]:
import numpy as np
dff = pd.DataFrame(columns=['aid','day'])
dff['aid'] = np.repeat(AIDS,28).astype('int32')
dff['day'] = np.tile(DAYS,len(AIDS)).astype('int8')

In [35]:
dff = dff.set_index( ['aid','day'] )

In [36]:
dff

Empty DataFrame
Columns: []
Index: [(1517085, 0), (1517085, 1), (1517085, 2), (1517085, 3), (1517085, 4), (1517085, 5), (1517085, 6), (1517085, 7), (1517085, 8), (1517085, 9), (1517085, 10), (1517085, 12), (1517085, 13), (1517085, 14), (1517085, 15), (1517085, 16), (1517085, 17), (1517085, 18), (1517085, 19), (1517085, 20), (1517085, 21), (1517085, 22), (1517085, 23), (1517085, 24), (1517085, 25), (1517085, 26), (1517085, 27), (1517085, 11), (1563459, 0), (1563459, 1), (1563459, 2), (1563459, 3), (1563459, 4), (1563459, 5), (1563459, 6), (1563459, 7), (1563459, 8), (1563459, 9), (1563459, 10), (1563459, 12), (1563459, 13), (1563459, 14), (1563459, 15), (1563459, 16), (1563459, 17), (1563459, 18), (1563459, 19), (1563459, 20), (1563459, 21), (1563459, 22), (1563459, 23), (1563459, 24), (1563459, 25), (1563459, 26), (1563459, 27), (1563459, 11), (1309446, 0), (1309446, 1), (1309446, 2), (1309446, 3), (1309446, 4), (1309446, 5), (1309446, 6), (1309446, 7), (1309446, 8), (1309446, 9), (1309446, 10), (1309446, 12), (1309446, 13), (1309446, 14), (1309446, 15), (1309446, 16), (1309446, 17), (1309446, 18), (1309446, 19), (1309446, 20), (1309446, 21), (1309446, 22), (1309446, 23), (1309446, 24), (1309446, 25), (1309446, 26), (1309446, 27), (1309446, 11), (16246, 0), (16246, 1), (16246, 2), (16246, 3), (16246, 4), (16246, 5), (16246, 6), (16246, 7), (16246, 8), (16246, 9), (16246, 10), (16246, 12), (16246, 13), (16246, 14), (16246, 15), (16246, 16), ...]

[3139332 rows x 0 columns]

In [37]:
dff = dff.merge(tmp, left_index=True, right_index=True, how='left').fillna(0).reset_index()

In [38]:
dff.head()

,aid,day,item,user
0,1517085,0,1.0,1.0
1,1517085,1,0.0,0.0
2,1517085,2,0.0,0.0
3,1517085,3,0.0,0.0
4,1517085,4,0.0,0.0


这里实际上是构造了一个矩阵

In [39]:
item_features3 = dff.groupby('aid').agg({'item':'std','user':'std'}).fillna(-1)
item_features3.columns = ['item_day_std','user_day_std']

f32 = ['item_day_std','user_day_std']
for c in f32: item_features3[c] = item_features3[c].astype('float32')

In [40]:
del dff
gc.collect()

0

In [41]:
item_features3.head()

,item_day_std,user_day_std
aid,,
4,0.416270,0.416270
38,0.314970,0.314970
53,1.700840,1.700840
114,0.262265,0.262265
116,0.188982,0.188982


# 	item_day_std，user_day_std

item_features3 = dff.groupby('aid').agg({'item':'std','user':'std'}).fillna(-1)
item_features3.columns = ['item_day_std','user_day_std']

f32 = ['item_day_std','user_day_std']
for c in f32: item_features3[c] = item_features3[c].astype('float32')

提取出关于每个 aid 对应的 item 和 user 列方差的信息，用作后续的特征工程和模型建模中。

In [42]:
df['mn'] = df.groupby('session').ts.transform('min')
df

,Unnamed: 0,session,aid,ts,type,day,mn
0,0,0,1517085,1659304800,0,0,1659304800
1,1,0,1563459,1659304904,0,0,1659304800
2,2,0,1309446,1659367439,0,0,1659304800
3,3,0,16246,1659367719,0,0,1659304800
4,4,0,1781822,1659367871,0,0,1659304800
...,...,...,...,...,...,...,...
299995,299995,4138,1035486,1660993454,0,19,1659304916
299996,299996,4138,1035486,1661016076,0,19,1659304916
299997,299997,4138,1035486,1661016271,0,19,1659304916
299998,299998,4138,972847,1661174639,0,21,1659304916


In [43]:
#检查一个时间是否大于3day
#df = df.loc[df['mn'] > mn+60*60*24*3 ]
df.shape
#去掉了最远的三天的数据可以筛选掉旧的、无用的数据记录，减小数据集的大小，提升计算效率和分析准确度。
#同时，也能更好地反映出用户的实际行为和偏好，避免旧数据对分析结果的干扰。

(300000, 7)

In [44]:
df['t'] = df.ts - df.mn
df

,Unnamed: 0,session,aid,ts,type,day,mn,t
0,0,0,1517085,1659304800,0,0,1659304800,0
1,1,0,1563459,1659304904,0,0,1659304800,104
2,2,0,1309446,1659367439,0,0,1659304800,62639
3,3,0,16246,1659367719,0,0,1659304800,62919
4,4,0,1781822,1659367871,0,0,1659304800,63071
...,...,...,...,...,...,...,...,...
299995,299995,4138,1035486,1660993454,0,19,1659304916,1688538
299996,299996,4138,1035486,1661016076,0,19,1659304916,1711160
299997,299997,4138,1035486,1661016271,0,19,1659304916,1711355
299998,299998,4138,972847,1661174639,0,21,1659304916,1869723


In [45]:
df = df.loc[df.t < 60*60*24*7]
df.shape
#时间差信息通常涉及到用户的活动时长和频率等重要指标。
#通过筛选出时间差小于 7 天的数据行，可以更准确地评估用户的活动情况和行为习惯，避免过期和无关的数据干扰分析结果。

(137746, 8)

In [46]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique'})
item_features.columns = ['buy_ratio','count_item','count_user']

In [47]:
item_features

,buy_ratio,count_item,count_user
aid,,,
38,0.0,3,2
114,0.0,1,1
137,0.0,1,1
183,0.0,1,1
219,0.0,1,1
...,...,...,...
1855500,0.0,5,3
1855547,0.0,1,1
1855584,0.0,1,1


In [48]:
item_features2 = df.groupby('aid').agg({'type':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std']

f32 = ['buy_ratio_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')
    #表示每个商品 aid 的购买比例的标准差。通过计算标准差，我们可以更好地描述每个商品购买比例的离散程度和波动幅度。

# buy_ratio_std
item_features2 = df.groupby('aid').agg({'type':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std']
表示每个商品 aid 的购买比例的标准差。通过计算标准差，我们可以更好地描述每个商品购买比例的离散程度和波动幅度。

In [49]:
item_features2

,buy_ratio_std
aid,
38,0.000000
114,-1.000000
137,-1.000000
183,-1.000000
219,-1.000000
...,...
1855500,0.000000
1855547,-1.000000
1855584,-1.000000


In [50]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [53]:
item_features = pd.concat([item_features,item_features2,item_features3],axis=1)

In [54]:
item_features.columns = [f'{c}_5' for c in item_features.columns]

In [55]:
item_features.head()

,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,0.416270,0.416270
38,0.0,3.0,2.0,1.5,0.0,0.314970,0.314970
53,NaN,NaN,NaN,NaN,NaN,1.700840,1.700840
114,0.0,1.0,1.0,1.0,-1.0,0.262265,0.262265
116,NaN,NaN,NaN,NaN,NaN,0.188982,0.188982


In [56]:
item_features.dtypes

buy_ratio_5        float32
count_item_5       float64
count_user_5       float64
repeat_5           float32
buy_ratio_std_5    float32
item_day_std_5     float32
user_day_std_5     float32
dtype: object

In [59]:
item_features.to_parquet('item12.pqt')